In [13]:
!pip install catboost

In [12]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from transformers import T5Tokenizer, T5EncoderModel
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert "ANSWER" column to T5 embeddings and return the dataset
def convert_to_t5_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Initialize T5 encoder model and tokenizer
    model = T5EncoderModel.from_pretrained('t5-small')
    tokenizer = T5Tokenizer.from_pretrained('t5-small')

    # Function to get T5 embeddings for a text
    def get_t5_embeddings(text):
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs).last_hidden_state
        return outputs.mean(dim=1).squeeze().numpy()

    # Generate embeddings for the "ANSWER" column
    embeddings = df[text_column].apply(lambda x: get_t5_embeddings(str(x)))

    # Stack embeddings into a 2D array
    embeddings = np.stack(embeddings.values)

    # Print the dimensions of the embeddings
    print(f"T5 embeddings dimensions for '{text_column}': {embeddings.shape}")

    # Return the embeddings and the original DataFrame
    return embeddings, df

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Define individual model functions
def knn_model(X_train, y_train, X_test, y_test):
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    model = MLPClassifier(max_iter=300)
    model.fit(X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    model = XGBClassifier(eval_metric='mlogloss')
    model.fit(X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    model = AdaBoostClassifier(algorithm='SAMME')
    model.fit(X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    model = CatBoostClassifier(verbose=0)
    model.fit(X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=300)
    model.fit(X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    model = SVC()
    model.fit(X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Specify the text column for T5 embeddings
text_column = 'answer'

# Convert text column to T5 embeddings and get the original DataFrame
embeddings, original_data = convert_to_t5_embeddings(input_file, text_column)

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
T5 embeddings dimensions for 'answer': (681, 512)
Running KNN model...
KNN Model Metrics:
Train Accuracy: 0.7710
Train Precision: 0.7622
Train Recall: 0.7710
Train F1 Score: 0.7643
Train Confusion Matrix:
[[ 17  10   5]
 [ 12  70  50]
 [  4  28 280]]

Test Accuracy: 0.7220
Test Precision: 0.7328
Test Recall: 0.7220
Test F1 Score: 0.7271
Test Confusion Matrix:
[[  3   7   3]
 [  6  18  17]
 [  6  18 127]]

Running Random Forest model...
Random Forest Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7512
Test Precision: 0.6657
Test Recall: 0.7512
Test F1 Score: 0.6755
Test Confusion Matrix:
[[  0   3  10]
 [  1   5  35]
 [  0   2 149]]

Running Decision Tree model...
Decision Tree Model Metrics:
Train Accuracy: 0.9937


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9938
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 129   3]
 [  0   0 312]]

Test Accuracy: 0.7073
Test Precision: 0.7253
Test Recall: 0.7073
Test F1 Score: 0.7153
Test Confusion Matrix:
[[  3   4   6]
 [  6  20  15]
 [  5  24 122]]

Running XGBoost model...
XGBoost Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9937
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   1 311]]

Test Accuracy: 0.7707
Test Precision: 0.7390
Test Recall: 0.7707
Test F1 Score: 0.7437
Test Confusion Matrix:
[[  2   4   7]
 [  2  15  24]
 [  0  10 141]]

Running AdaBoost model...
AdaBoost Model Metrics:
Train Accuracy: 0.7689
Train Precision: 0.7802
Train Recall: 0.7689
Train F1 Score: 0.7712
Train Confusion Matrix:
[[ 19   9   4]
 [  1  87  44]
 [  0  52 260]]

Test Accuracy: 0.6341
Test Precision: 0.6697
Test Recall: 0.6341
Test F1 Score: 0.6